<h1> Evaluating RAG </h1>

<p> In this notebook we evaluate the Faithfulness and Relevancy of the RAG + Vector Store set up in the previous notebook. We test this across various LLM models available in Ollama </p>

<h3> Library Imports </h3>

In [15]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import json
import logging
import sys
import llama_index
import nest_asyncio

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
llama_index.set_global_handler("simple")
nest_asyncio.apply()

<h3> Connect LLM, Vector Stores (Start with Mistral) </h3>

In [62]:
from llama_index.embeddings import VoyageEmbedding
import os
from qdrant_client import QdrantClient
from llama_index.llms import Ollama

model_name = "voyage-2"
api_key = ""
voyage_api_key = os.environ.get("VOYAGE_API_KEY", api_key)

embed_model = VoyageEmbedding(
    model_name=model_name, voyage_api_key=voyage_api_key, 
)


llm = Ollama(model="mistral", request_timeout=30.0)

client = QdrantClient("localhost", port=6333)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/tylerroth/anaconda3/lib/python3.11/site-packages/certifi/cacert.pem'
load_verify_locations cafile='/Users/tylerroth/anaconda3/lib/python3.11/site-packages/certifi/cacert.pem'


In [157]:
from llama_index import set_global_service_context
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model
)

set_global_service_context(service_context)

In [158]:
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index import VectorStoreIndex

vector_store = QdrantVectorStore(client=client, collection_name="Naive_RAG") # name the vector store 
storage_context = StorageContext.from_defaults(vector_store=vector_store) # connect llamaindex with qdrant
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)
query_engine = index.as_query_engine()



DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=6333 local_address=None timeout=5.0 socket_options=None
connect_tcp.started host='localhost' port=6333 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x1734f9250>
connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x1734f9250>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'GET']>
send_request_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_headers.complete
send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'GET']>
send_request_body.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_body.complete
send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'GET']>
receive_response_headers.started request=<Request [b'GET']>
DEBU

<h3> Load in Evaluation Questions </h3>

In [5]:
import pandas as pd 
sampled_questions = pd.read_csv('Sampled_Questions.csv')['SampleQuestions'].to_list()

In [6]:
sampled_questions

['What actions did protesters take during their demonstrations?',
 'What commodities does Gangavaram Port handle?',
 'What historic sacrifices is New Zealand asking its citizens to make during the COVID-19 crisis, according to the text?',
 'Which social media platform did Kangana use to post the childhood photos and memories on Siblings Day 2021?',
 "How many Tests and ODIs did Anju Jain play for the Indian Women's team?",
 'What are some of the most notable controversies involving Salman Khan during his film career?',
 'What was the outcome of the June 1, 2020, protest that led to the large number of arrests?',
 'What was Chris Farley known for in the entertainment industry?',
 "How did CSK's captain feel about his team's bowling performance?",
 'In which city were the second-highest number of home units sold during the first quarter of 2021?',
 'Why is Rimpa Chaudhary known for being friendly with her brides and listening to their needs?',
 'What advice did ministers give to under-60

<h3> Showing Faithfulness Evaluation </h3>
<p> Faithfulness is akin to hallucination, it measures whether or not the LLM is making up information that is not available in the context given to it. </p>

In [7]:
from llama_index import Response

def display_eval_df(response: Response, eval_result: str) -> None:
    if response.source_nodes == []:
        print("no response!")
        return
    eval_df = pd.DataFrame(
        {
            "Response": str(response),
            "Source": response.source_nodes[0].node.text[:1000] + "...",
            "Evaluation Result": "Pass" if eval_result.passing else "Fail",
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

def pretty_eval_df(response: Response, eval_result: str) -> None:
    if response.source_nodes == []:
        print("no response!")
        return
    eval_df = pd.DataFrame(
        {
            "Response": str(response),
            "Source": response.source_nodes[0].node.text[:1000] + "...",
            "Evaluation Result": "Pass" if eval_result.passing else "Fail",
        },
        index=[0],
    )
    
    return(eval_df)

In [52]:
from llama_index.evaluation import FaithfulnessEvaluator

response = query_engine.query(sampled_questions[3])
evaluator = FaithfulnessEvaluator(service_context=service_context)
eval_result = evaluator.evaluate_response(response=response)
print(str(eval_result.passing))

DEBUG:voyage:message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
DEBUG:voyage:api_version=None data='{"input": ["Which social media platform did Kangana use to post the childhood photos and memories on Siblings Day 2021?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
api_version=None data='{"input": ["Which social media platform did Kangana use to post the childhood photos and memories on Siblings Day 2021?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.voyageai.com:443 "POST /v1/embeddings HTTP/1.1" 200 None
https://api.voyageai.com:443 "POST /v1/embeddings HTTP/1.1" 200 None
DEBUG:voyage:message='Voyage API response' path=https://api.voyageai.com/v1/embeddings processing_ms=None request_id=None response_code=200
messa

<h3> Showing Relevancy Evaluation </h3>
<p> Relevancy measures the contextual similarity between a given prompt, response, and the articles retrieved from the vector store. </p>

In [20]:
from llama_index.evaluation import RelevancyEvaluator

evaluator = RelevancyEvaluator(service_context=service_context)
response = query_engine.query(sampled_questions[3])
eval_result = evaluator.evaluate_response(query=sampled_questions[3], response=response)
print(str(eval_result))

DEBUG:voyage:message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
DEBUG:voyage:api_version=None data='{"input": ["Which social media platform did Kangana use to post the childhood photos and memories on Siblings Day 2021?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
api_version=None data='{"input": ["Which social media platform did Kangana use to post the childhood photos and memories on Siblings Day 2021?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.voyageai.com:443
Starti

<h3> Mistral Tests </h3>

<p> Lets only assess 50 samples. </p>

In [23]:
test_sample = sampled_questions[:50]

In [24]:
len(test_sample)

50

<h5> Faithfulness </h5>

In [25]:
from llama_index.evaluation import FaithfulnessEvaluator
import time

max_retries = 3  # Set the maximum number of retries
retry_delay = 3  # Set the delay between retries in seconds

faithfulness_df_list_mistral = []
evaluator = FaithfulnessEvaluator(service_context=service_context)

for i in test_sample:
    retries = 0
    while retries < max_retries:
        try:
            response = query_engine.query(i)
            eval_result = evaluator.evaluate_response(response=response)
            faithfulness_df = pretty_eval_df(response, eval_result)
            faithfulness_df_list_mistral.append(faithfulness_df)
            break  # If the query is successful, exit the loop
        except Exception as e:
            print(f"An error occurred: {e}")
            retries += 1
            if retries == max_retries:
                print(f"Max retries reached for query: {i}")
                break
            time.sleep(retry_delay)  # Wait before the next retry


DEBUG:voyage:message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
DEBUG:voyage:api_version=None data='{"input": ["What actions did protesters take during their demonstrations?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
api_version=None data='{"input": ["What actions did protesters take during their demonstrations?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.voyageai.com:443
Starting new HTTPS connection (1): api.voyageai.com:443
DEBUG:urllib3.connectionpool:https://api.v

In [26]:
master_faithfull_mistral = pd.concat(faithfulness_df_list_mistral)
master_faithfull_mistral['Evaluation Result'].value_counts()

Evaluation Result
Pass    44
Fail     6
Name: count, dtype: int64

<h3> Relevancy </h3>

In [64]:
from llama_index.evaluation import RelevancyEvaluator
import time

max_retries = 3  # Set the maximum number of retries
retry_delay = 3  # Set the delay between retries in seconds

relevancy_df_list_mistral = []
evaluator = FaithfulnessEvaluator(service_context=service_context)

for i in test_sample:
    retries = 0
    while retries < max_retries:
        try:
            response = query_engine.query(i)
            eval_result = evaluator.evaluate_response(query=i, response=response)
            relevancy_df = pretty_eval_df(response, eval_result)
            relevancy_df_list_mistral.append(relevancy_df)
            break  # If the query is successful, exit the loop
        except Exception as e:
            print(f"An error occurred: {e}")
            retries += 1
            if retries == max_retries:
                print(f"Max retries reached for query: {i}")
                break
            time.sleep(retry_delay)  # Wait before the next retry


DEBUG:voyage:message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
DEBUG:voyage:api_version=None data='{"input": ["What actions did protesters take during their demonstrations?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
api_version=None data='{"input": ["What actions did protesters take during their demonstrations?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.voyageai.com:443
Starting new HTTPS connection (1): api.voyageai.com:443
DEBUG:urllib3.connectionpool:https://api.v

In [65]:
master_relevancy_mistral = pd.concat(relevancy_df_list_mistral)
master_relevancy_mistral['Evaluation Result'].value_counts()

Evaluation Result
Pass    43
Fail     7
Name: count, dtype: int64

<h3> Testing Llama2 </h3>

In [70]:
llm = Ollama(model="llama2", request_timeout=30.0)

service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model
)

set_global_service_context(service_context)

<h5> Faithfulness </h5>

In [38]:
max_retries = 3  # Set the maximum number of retries
retry_delay = 3  # Set the delay between retries in seconds

faithfulness_df_list_llama = []
evaluator = FaithfulnessEvaluator(service_context=service_context)

for i in test_sample:
    retries = 0
    while retries < max_retries:
        try:
            response = query_engine.query(i)
            eval_result = evaluator.evaluate_response(response=response)
            faithfulness_df = pretty_eval_df(response, eval_result)
            faithfulness_df_list_llama.append(faithfulness_df)
            break  # If the query is successful, exit the loop
        except Exception as e:
            print(f"An error occurred: {e}")
            retries += 1
            if retries == max_retries:
                print(f"Max retries reached for query: {i}")
                break
            time.sleep(retry_delay)  # Wait before the next retry

DEBUG:voyage:message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
DEBUG:voyage:api_version=None data='{"input": ["What actions did protesters take during their demonstrations?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
api_version=None data='{"input": ["What actions did protesters take during their demonstrations?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.voyageai.com:443
Starting new HTTPS connection (1): api.voyageai.com:443
DEBUG:urllib3.connectionpool:https://api.v

In [39]:
master_faithfull_llama = pd.concat(faithfulness_df_list_llama)
master_faithfull_llama['Evaluation Result'].value_counts()

Evaluation Result
Pass    46
Fail     4
Name: count, dtype: int64

<h3> Relevancy </h3>

In [72]:
from llama_index.evaluation import RelevancyEvaluator
import time

max_retries = 3  # Set the maximum number of retries
retry_delay = 3  # Set the delay between retries in seconds

relevancy_df_list_llama = []
evaluator = FaithfulnessEvaluator(service_context=service_context)

for i in test_sample:
    retries = 0
    while retries < max_retries:
        try:
            response = query_engine.query(i)
            eval_result = evaluator.evaluate_response(query=i, response=response)
            relevancy_df = pretty_eval_df(response, eval_result)
            relevancy_df_list_llama.append(relevancy_df)
            break  # If the query is successful, exit the loop
        except Exception as e:
            print(f"An error occurred: {e}")
            retries += 1
            if retries == max_retries:
                print(f"Max retries reached for query: {i}")
                break
            time.sleep(retry_delay)  # Wait before the next retry


DEBUG:voyage:message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
DEBUG:voyage:api_version=None data='{"input": ["What actions did protesters take during their demonstrations?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
api_version=None data='{"input": ["What actions did protesters take during their demonstrations?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.voyageai.com:443
Starting new HTTPS connection (1): api.voyageai.com:443
DEBUG:urllib3.connectionpool:https://api.v

In [73]:
master_relevancy_llama = pd.concat(relevancy_df_list_llama)
master_relevancy_llama['Evaluation Result'].value_counts()

Evaluation Result
Pass    45
Fail     5
Name: count, dtype: int64

<h3> Testing Dolphin-Phi </h3>

In [51]:
llm = Ollama(model="dolphin-phi", request_timeout=30.0)

service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model
)

set_global_service_context(service_context)

<h3> Faithfulness </h3>

In [53]:
max_retries = 3  # Set the maximum number of retries
retry_delay = 3  # Set the delay between retries in seconds

faithfulness_df_list_dphi = []
evaluator = FaithfulnessEvaluator(service_context=service_context)

for i in test_sample:
    retries = 0
    while retries < max_retries:
        try:
            response = query_engine.query(i)
            eval_result = evaluator.evaluate_response(response=response)
            faithfulness_df = pretty_eval_df(response, eval_result)
            faithfulness_df_list_dphi.append(faithfulness_df)
            break  # If the query is successful, exit the loop
        except Exception as e:
            print(f"An error occurred: {e}")
            retries += 1
            if retries == max_retries:
                print(f"Max retries reached for query: {i}")
                break
            time.sleep(retry_delay)  # Wait before the next retry

DEBUG:voyage:message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
DEBUG:voyage:api_version=None data='{"input": ["What actions did protesters take during their demonstrations?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
api_version=None data='{"input": ["What actions did protesters take during their demonstrations?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.voyageai.com:443 "POST /v1/embeddings HTTP/1.1" 200 None
https://api.voyageai.com:443 "POST /v1/embeddings HTTP/1.1" 200 None
DEBUG:voyage:message='Voyage API response' path=https://api.voyageai.com/v1/embeddings processing_ms=None request_id=None response_code=200
message='Voyage API response' path=https://api.voyageai.com/v1/embeddings processing_ms=None requ

In [54]:
master_faithfull_dphi = pd.concat(faithfulness_df_list_dphi)
master_faithfull_dphi['Evaluation Result'].value_counts()

Evaluation Result
Fail    34
Pass    16
Name: count, dtype: int64

<h3> Relevancy </h3>

In [60]:
from llama_index.evaluation import RelevancyEvaluator
import time

max_retries = 3  # Set the maximum number of retries
retry_delay = 3  # Set the delay between retries in seconds

relevancy_df_list_dphi = []
evaluator = FaithfulnessEvaluator(service_context=service_context)

for i in test_sample:
    retries = 0
    while retries < max_retries:
        try:
            response = query_engine.query(i)
            eval_result = evaluator.evaluate_response(query=i, response=response)
            relevancy_df = pretty_eval_df(response, eval_result)
            relevancy_df_list_dphi.append(relevancy_df)
            break  # If the query is successful, exit the loop
        except Exception as e:
            print(f"An error occurred: {e}")
            retries += 1
            if retries == max_retries:
                print(f"Max retries reached for query: {i}")
                break
            time.sleep(retry_delay)  # Wait before the next retry


DEBUG:voyage:message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
DEBUG:voyage:api_version=None data='{"input": ["What actions did protesters take during their demonstrations?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
api_version=None data='{"input": ["What actions did protesters take during their demonstrations?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.voyageai.com:443
Starting new HTTPS connection (1): api.voyageai.com:443
DEBUG:urllib3.connectionpool:https://api.v

In [61]:
master_relevancy_dphi = pd.concat(relevancy_df_list_dphi)
master_relevancy_dphi['Evaluation Result'].value_counts()

Evaluation Result
Fail    31
Pass    19
Name: count, dtype: int64

<h3> Evaluating if RAG would find the correct article when asked about more recent news (2024) </h3>

In [148]:
llm = Ollama(model="llama2", request_timeout=30.0)

service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model
)

set_global_service_context(service_context)

In [149]:
sampled_questions

['What actions did protesters take during their demonstrations?',
 'What commodities does Gangavaram Port handle?',
 'What historic sacrifices is New Zealand asking its citizens to make during the COVID-19 crisis, according to the text?',
 'Which social media platform did Kangana use to post the childhood photos and memories on Siblings Day 2021?',
 "How many Tests and ODIs did Anju Jain play for the Indian Women's team?",
 'What are some of the most notable controversies involving Salman Khan during his film career?',
 'What was the outcome of the June 1, 2020, protest that led to the large number of arrests?',
 'What was Chris Farley known for in the entertainment industry?',
 "How did CSK's captain feel about his team's bowling performance?",
 'In which city were the second-highest number of home units sold during the first quarter of 2021?',
 'Why is Rimpa Chaudhary known for being friendly with her brides and listening to their needs?',
 'What advice did ministers give to under-60

In [166]:
response = query_engine.query("What is going on with Nvidia's stock right now?")

DEBUG:voyage:message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
message='Request to Voyage API' method=post path=https://api.voyageai.com/v1/embeddings
DEBUG:voyage:api_version=None data='{"input": ["What is going on with Nvidia\'s stock right now?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
api_version=None data='{"input": ["What is going on with Nvidia\'s stock right now?"], "model": "voyage-2", "input_type": "query", "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.voyageai.com:443 "POST /v1/embeddings HTTP/1.1" 200 None
https://api.voyageai.com:443 "POST /v1/embeddings HTTP/1.1" 200 None
DEBUG:voyage:message='Voyage API response' path=https://api.voyageai.com/v1/embeddings processing_ms=None request_id=None response_code=200
message='Voyage API response' path=https://api.voyageai.com/v1/embeddings processing_ms=None request_id=None response_code=

In [167]:
print(response.response)

Nvidia's stock is currently experiencing a significant increase in value, with a 15% jump in premarket trading after the company reported bumper earnings that beat Wall Street estimates. The U.S. tech giant posted revenue of $22.10 billion for its fiscal fourth quarter, a rise of 265% year on year, while net income surged by 769%. The company's Data Center business, which includes its H100 graphics cards used for AI training, posted sales of $18.4 billion in the fourth quarter, representing 409% year-on-year growth. The positive outlook from Nvidia has prompted a round of broker upgrades, with JPMorgan raising its price target on the stock from $650 to $850 and Bank of America Global Research hiking its target from $800 to $925.


In [168]:
print(response.metadata)

{'a7667cec-810d-4063-a354-4d565ab67926': {'website': 'www.cnbc.com', 'section': 'Tech', 'sub_section': '', 'article_url': 'https://www.cnbc.com/2024/02/22/nvidia-shares-pop-12percent-in-premarket-trade-after-ai-fueled-bumper-earnings-.html', 'title': 'Nvidia shares pop 15% after AI-fueled bumper earnings', 'published_date': '2024-02-22', 'user_agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1', 'language': ''}, 'e24969e3-67e6-4ebc-ad77-1d8d3c440f8b': {'website': 'www.deccanchronicle.com', 'section': 'Top Stories', 'sub_section': '', 'article_url': 'https://www.deccanchronicle.com/business/companies/260321/tata-stocks-raise-after-supreme-court-verdict.html', 'title': 'TATA stocks raise after Supreme Court verdict', 'published_date': 'Mar 27, 2021, 4:49 am IST', 'language': ''}}
